# 功課三 - Create a NB Classifier
* use the dictionary created from HW2
* 13 classes (id 1~13), each class has 15 training documents (195)
* calculate M for each class
    * calculate with add-one smoothing   
* feature selection - 500 terms 

## Import

In [1]:
# import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download('stopwords')

In [2]:
import numpy as np
import pandas as pd
from collections import Counter
import pickle
import math
import glob
import os

In [3]:
from nltk.stem import PorterStemmer
p = PorterStemmer()
stemmed_doc =[]

In [4]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

## Create train test

In [5]:
!pwd

/Users/stephanie/Desktop/2022_Information_Retrieve/HW3


In [6]:
os.chdir('..')
os.chdir('./Data/IRTM/')
my_files = glob.glob('*.txt')

# /Users/stephanie/Desktop/2022_Information_Retrieve/HW3

In [7]:
my_dict={}

punctuations = '''!()-[]{};:'`"\,<>./?@#$%^&*_~'''
numbers ='1234567890'

for file in my_files:

    with open(file) as f:
        lines = f.read()
    file_num = file.strip('.txt')
    
    # 處理標點符號與大小寫, 句子沒有斷乾淨
    for punc in punctuations:
        lines = lines.replace(punc, ' ').replace('\n','')
    for num in numbers:
        lines = lines.replace(num,'')
        
    doc_list = lines.lower().split(' ')
    
    stemmed_doc = [p.stem(token) for token in doc_list if p.stem(token)!= '\n' and not p.stem(token).isnumeric() and len(p.stem(token))>1 ]
    
    
    final_tokens={}
    for word in stemmed_doc:
        if word not in stop_words and not word.isnumeric() and len(word)>1:
            if word not in final_tokens:
                final_tokens[word] = 1
            else:
                final_tokens[word] += 1


    my_dict[file_num] ={}
    my_dict[file_num]['tokens'] = final_tokens
    my_dict[file_num]['label'] = ''
    
    # for token in final_tokens:
    #     if token in my_dict:
    #         my_dict[token]+=1
    #     else:
    #         my_dict[token]=1

In [8]:
len(my_dict)

1095

In [9]:
!pwd

/Users/stephanie/Desktop/2022_Information_Retrieve/Data/IRTM


In [10]:
os.chdir('..')
os.chdir('..')

text =[]
file_path = './HW3/training.txt'
with open(file_path) as f:
    lines = f.read()
    text.append(lines)
print(text)


['1 11 19 29 113 115 169 278 301 316 317 321 324 325 338 341 \n2 1 2 3 4 5 6 7 8 9 10 12 13 14 15 16 \n3 813 817 818 819 820 821 822 824 825 826 828 829 830 832 833 \n4 635 680 683 702 704 705 706 708 709 719 720 722 723 724 726 \n5 646 751 781 794 798 799 801 812 815 823 831 839 840 841 842 \n6 995 998 999 1003 1005 1006 1007 1009 1011 1012 1013 1014 1015 1016 1019 \n7 700 730 731 732 733 735 740 744 752 754 755 756 757 759 760 \n8 262 296 304 308 337 397 401 443 445 450 466 480 513 533 534 \n9 130 131 132 133 134 135 136 137 138 139 140 141 142 143 145 \n10 31 44 70 83 86 92 100 102 305 309 315 320 326 327 328 \n11 240 241 243 244 245 248 250 254 255 256 258 260 275 279 295 \n12 535 542 571 573 574 575 576 578 581 582 583 584 585 586 588 \n13 485 520 523 526 527 529 530 531 532 536 537 538 539 540 541 ']


In [11]:
train_dict = {}
test_dict = {}
term_dict = {}
labels = []
# train
for i in text[0].split('\n'):
    cnt=0
    for j in i.split(' '):
        if len(j)>0:
            if cnt == 0 :
                cnt+=1
                label = j
                labels.append(label)
            else:
                my_dict[j]['label']=label
                train_dict[j]=my_dict[j]

#test 
for i in my_dict:
    if i not in train_dict:
        test_dict[i]=my_dict[i]


In [12]:
print(len(train_dict) , len(test_dict))

195 900


In [13]:
train_dict

{'11': {'tokens': {'navi': 8,
   'unit': 2,
   'state': 2,
   'japan': 7,
   'south': 5,
   'korea': 4,
   'singapor': 6,
   'monday': 2,
   'began': 1,
   'first': 2,
   'combin': 1,
   'submarin': 14,
   'rescu': 10,
   'exercis': 10,
   'pacif': 5,
   'day': 1,
   'reach': 2,
   'involv': 1,
   'peopl': 2,
   'four': 2,
   'ship': 2,
   'three': 1,
   'sophist': 1,
   'underwat': 3,
   'devic': 2,
   'personnel': 1,
   'distress': 1,
   'joint': 1,
   'china': 2,
   'sea': 1,
   'wa': 2,
   'plan': 1,
   'nearli': 1,
   'two': 1,
   'year': 2,
   'befor': 1,
   'recent': 2,
   'russian': 2,
   'disast': 2,
   'kill': 1,
   'particip': 1,
   'said': 6,
   'heighten': 1,
   'import': 2,
   'thi': 2,
   'time': 1,
   'realli': 1,
   'chanc': 1,
   'work': 2,
   'togeth': 2,
   'region': 2,
   'lt': 1,
   'lesli': 1,
   'hull': 2,
   'ryde': 2,
   'spokeswoman': 1,
   'promot': 1,
   'greater': 1,
   'understand': 1,
   'also': 2,
   'commit': 1,
   'stabil': 1,
   'asia': 1,
   'ha': 2

## Feature Selection

In [14]:
def extract_vocabulary(token_lists) -> list:
    return {token for idx in token_lists for token in token_lists[idx].get('tokens')}

In [50]:
def extract_by_categories(token_lists) -> dict:
    grouped_tokens = {}
    
    for idx in token_lists:
        if token_lists[idx]['label'] not in grouped_tokens:
            grouped_tokens[token_lists[idx]['label']] = { token  for token in token_lists[idx].get('tokens')}
        else:
            for token in token_lists[idx].get('tokens'):
                if token not in grouped_tokens[token_lists[idx]['label']]:
                    grouped_tokens[token_lists[idx]['label']].update(token)


    return grouped_tokens

In [15]:
vocab_l = []
vocab_l = extract_vocabulary(train_dict)
print(f'before feature extraction :  {len(vocab_l)}')

before feature extraction :  5197


In [51]:
vocab_l = []
vocab_l = extract_by_categories(train_dict)
print(f'before feature extraction :  {len(vocab_l)}')

before feature extraction :  13


### Chi2

In [18]:
def chi_square(C: list, D: dict):
    vocabulary = extract_vocabulary(D)
    N = len(D)
    chi2 = dict()
    count = 0
    for term in vocabulary:
        chi2_term = 0
        matrix = dict()
        matrix["tp"] = [ idx for idx in D if term in D[idx]['tokens']]
        matrix["ta"] = [ idx for idx in D if term not in D[idx]['tokens']]

        for c in C:
            matrix["cp"] = [ idx for idx in D if c ==  D[idx]['label']]
            matrix["ca"] = [ idx for idx in D if c !=  D[idx]['label']]
            matrix["tp_cp"] = len([idx for idx in matrix['tp'] if D[idx]['label']== int(c) ])
            matrix["tp_ca"] = len([idx for idx in matrix['tp'] if D[idx]['label']!= int(c) ])
            matrix["ta_cp"] = len([idx for idx in matrix['ta'] if D[idx]['label']== int(c) ])
            matrix["ta_ca"] = len([idx for idx in matrix['ta'] if D[idx]['label']!= int(c) ])

            chi2_class = 0
            for i in ["tp", "ta"]:
                for j in ["cp", "ca"]:
                    E = (len(matrix[i]) * len(matrix[j]) )/ N
                    
                    chi2_class += ((matrix[f"{i}_{j}"] - E)**2) / E
            chi2_term += chi2_class
        chi2[term] = chi2_term
    # 最終需要的 500 字
    vocabulary = sorted(chi2, key=chi2.get, reverse=True)[:500] 
    return vocabulary 

In [19]:
vocab_l = chi_square(labels,train_dict)
print(f'after feature extraction : {len(vocab_l)}')

after feature extraction : 500


### Log-likelihood

In [55]:
def log_likelihood(C: list, D: dict):

    # get all the terms in train dict
    vocabulary = extract_vocabulary(D)
    grouped_token = extract_by_categories(D)
    N = len(D)
    log_ll = dict()
    count = 0

    for tok in vocabulary:

        # go through all the classes
        for c in C :
            llr_l = []
            n11=0 
            n10=0 
            n01=0 
            n00=0

            # in topic
            if tok in grouped_token[c]:
                n11+=1
            else:
                n10+=1
            
            # other topics
            for j in grouped_token:
                if j!= c:
                    if tok in grouped_token[j]:
                        n01+=1
                    else:
                        n00+=1
            N = n11+n10+n01+n00
            pt= (n11+n01)/N #pt for Hypothesis 1
            p1= n11/(n11+n10) #p1 for Hypothesis 2
            p2= n01/(n01+n00) #p2 for Hypothesis 2
            LR=((pt**n11)*((1-pt)**n10)*(pt**n01)*((1-pt)**n00))/((p1**n11)*((1-p1)**n10)*(p2**n01)*((1-p2)**n00))
            LLR=(-2)*math.log2(LR)
            llr_l.append(LLR)
        log_ll[tok] = sum(llr_l)/len(llr_l)
    # 最終需要的 500 字
    vocabulary = sorted(log_ll, key=log_ll.get, reverse=True)[:500] 
    return vocabulary

In [56]:
vocab_l = log_likelihood(labels,train_dict)
print(f'after feature extraction : {len(vocab_l)}')

after feature extraction : 500


# Training

In [37]:
def train_multinomial_nb(C: list, D: dict, vocabulary):
    n_docs = len(D)
    prior = dict()
    cond_prob = {term: dict() for term in vocabulary}
    
    for c in C:
        n_class_docs = len([idx for idx in D if D[idx]['label']== c])
        class_docs = [idx for idx in D if D[idx]['label']== c]
        tct = dict()
        # 事前機率
        prior[c] = n_class_docs / n_docs
        
        # 事後機率
        for term in vocabulary:
            tokens_of_term = 0

            for idx in class_docs:
                if term in D[idx]['tokens']:
                    tokens_of_term += D[idx]['tokens'][term]
            tct[term] = tokens_of_term
        
        for term in vocabulary:
            cond_prob[term][c] = (tct[term]+1) / (sum(tct.values())+len(vocabulary))

            
    return vocabulary, prior, cond_prob

# Predict

In [39]:
def predict_multinomial_nb(D:dict, C:list, vocabulary, prior, cond_prob):
    
    result = {}
    for idx in D:
        max_val = 0
        predicts = []
        for c in C:
            score = math.log2(prior[c])
            for term in D[idx]['tokens']:
                if term in vocabulary:
                    score += (math.log2(cond_prob[term][c]))*(D[idx]['tokens'][term]) # 0.40
                    # score += (math.log2(cond_prob[term][c])) # 0.3788
            predicts.append(score)
        max_val = max(predicts)
        result.setdefault(str(idx),str(predicts.index(max_val)+1))
    return result


In [57]:
vocabulary, prior, cond_prob = train_multinomial_nb(labels , train_dict , vocab_l)

In [58]:
result = predict_multinomial_nb(test_dict , labels , vocabulary, prior, cond_prob)

In [59]:
# write file
with open('result_5.csv', 'w') as f:
	f.write("%s,%s\n"%('Id','Value'))
	for key in result.keys():
		f.write("%s,%s\n"%(key,result[key]))
f.close()

In [ ]:
# Score 紀錄
#  log likelihood 0.68
# chi-square 0.48